<a href="https://colab.research.google.com/github/Hydenx2004/AI-lawyer/blob/main/AI_Lawyer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install huggingface_hub

In [3]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)


In [5]:
llama_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="aboonaji/llama2finetune-v2",  # The path or name of the pre-trained model to load.
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,  # Load the model using 4-bit quantization to save memory and improve efficiency.
        bnb_4bit_compute_dtype=getattr(torch, "float16"),  # Use float16 data type for computation to balance precision and performance.
        bnb_4bit_quant_type="nf4"  # Use NF4 quantization type, which is a specific method of 4-bit quantization.
    )
)

# Disable caching mechanism
llama_model.config.use_cache = False  # Setting use_cache to False disables the caching of intermediate states during generation. This can save memory at the cost of potentially slower generation.

# Configure pretraining
llama_model.config.pretraining_tp = 1  # This parameter is used for specific pretraining configurations. Setting it to 1 could be a model-specific requirement or a way to control the training phases.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

In [6]:
# Load a pre-trained tokenizer for the LLaMA model
llama_tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path="aboonaji/llama2finetune-v2",  # The path or name of the pre-trained model to load the tokenizer from.
    trust_remote_code=True  # Allows loading custom code from the model repository if needed.
)

# Set the padding token to be the same as the end-of-sequence (EOS) token
llama_tokenizer.pad_token = llama_tokenizer.eos_token  # This sets the padding token to be the end-of-sequence token. Useful for models that do not have a dedicated pad token.

# Set the padding side to "right"
llama_tokenizer.padding_side = "right"  # Specifies that padding should be added to the right side of sequences. Ensures that all sequences are the same length when fed into the model.

# The tokenizer is now configured for use with the LLaMA model with appropriate padding settings.

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [7]:
# Define the training arguments for the model training process
training_arguments = TrainingArguments(
    output_dir="./results",  # The directory where the trained model and other outputs will be saved.
    per_device_train_batch_size=4,  # The batch size per GPU/TPU core/CPU during training. This defines how many samples will be processed at once per device.
    max_steps=100  # The total number of training steps to perform. Overrides the number of epochs if specified. Used for quick testing or debugging.
)

In [11]:
# Load the training dataset
train_dataset = load_dataset(path="viber1/indian-law-dataset", split="train")

# Define the SFT (Sequence-to-Sequence Fine-Tuning) Trainer for the LLaMA model
llama_sft_trainer = SFTTrainer(
    model=llama_model,  # The pre-trained model to be fine-tuned.
    args=training_arguments,  # The training arguments specifying how the training should be conducted.
    train_dataset=train_dataset,  # The dataset to be used for training.
    tokenizer=llama_tokenizer,  # The tokenizer to process the dataset.
    peft_config=LoraConfig(
        task_type="CAUSAL_LM",  # The type of task. "CAUSAL_LM" indicates a causal language modeling task.
        r=64,  # The rank of the low-rank adaptation matrices used in the LoRA method. Higher values might capture more information but increase computational cost.
        lora_alpha=16,  # A scaling factor for the LoRA method. It adjusts the influence of the low-rank matrices on the model's original parameters.
        lora_dropout=0.1  # Dropout rate for the LoRA method to prevent overfitting. Dropout is applied to the low-rank matrices.
    ),
    dataset_text_field="Response"  # The field in the dataset that contains the text data to be used for training.
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/24607 [00:00<?, ? examples/s]

In [12]:
llama_sft_trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=100, training_loss=1.9584205627441407, metrics={'train_runtime': 284.1449, 'train_samples_per_second': 1.408, 'train_steps_per_second': 0.352, 'total_flos': 1520032522567680.0, 'train_loss': 1.9584205627441407, 'epoch': 0.02})

In [13]:
user_prompt = input("Enter your prompt: ")
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 300)
model_answer = text_generation_pipeline(f"[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])

Enter your prompt: Can you explain the fundamental rights guaranteed by the Indian Constitution?


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST] Can you explain the fundamental rights guaranteed by the Indian Constitution? [/INST]  The Indian Constitution guarantees several fundamental rights to its citizens, which are essential for the well-being and dignity of individuals. everybody. These rights are enshrined in Part III of the Constitution and are considered fundamental to the governance of the country. Here are some of the fundamental rights guaranteed by the Indian Constitution:

1. Right to Life and Personal Liberty: Article 21 of the Constitution guarantees the right to life and personal liberty to every individual. This right includes the right to live with dignity, to move freely, and to pursue one's goals without interference from the state or any other authority.

2. Freedom of Speech and Expression: Article 19(1)(a) of the Constitution guarantees the right to freedom of speech and expression. This right includes the right to express one's opinions, ideas, and beliefs through words, writings, or any other mea